In [ ]:
import requests
from requests import Response
url = "https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json"
try:
    response:Response = requests.get(url)
    response_json = response.json()
    print(response.text)
except Exception as e:
    print(e)


In [1]:
import requests
from requests import Response
import csv
import pandas
import os
import sqlite3

In [ ]:
from pyproj import Transformer

# 創建轉換器
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")
twd97_x = 305689.109
twd97_y = 2768207.525
# 轉換 TWD97 到 WGS84
latitude,longitude = transformer.transform(twd97_x, twd97_y)

print(f'TWD97轉WGS84: Latitude: {latitude:.7}, Longitude: {longitude:.7}')


TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518


twd97是台灣的座標系統
WGS84是世界的座標系統
經度 longitude
緯度 latitude
所以台灣的X = longitude, Y = latitude

In [14]:
from geopy.geocoders import Nominatim

# 初始化 Nominatim 反向地理編碼器
geolocator = Nominatim(user_agent='PROJ')

def get_address_from_coordinates(lat, lon):
    """ 根據經緯度獲取地址或道路名稱 """
    location = geolocator.reverse((lat, lon), language='zh-TW', exactly_one=True)
    if location:
        return location.address
    return "未找到地址"

# 範例：查詢經緯度 (台北市)
latitude = 25.02066
longitude = 121.5518

address = get_address_from_coordinates(latitude, longitude)
print(address)
print(type(address))
print(f"座標 ({latitude}, {longitude}) 對應的地址是：{address}")


7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣
<class 'str'>
座標 (25.02066, 121.5518) 對應的地址是：7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣


In [47]:
address = "7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣"
list_address = [item.strip() for item in address.split(",")]
list_address.reverse()
print(list_address)
dic_address = {} 


['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']


In [23]:
display(type(response_json))
# print(response_json["features"][0]["properties"])
response_json01 = response_json["features"][0]["properties"]
for k,v in response_json01.items():
    print(f"{k}:{v}")

dict

BILL_CODE:10967113535908
URGENT_RCV_DATE:20241017
URGENT_START_DATE:20241116000000
URGENT_END_DATE:20241116220000
URGENT_ADDRESS1:台北市樂業街108巷1至11號
X1:305689.109
Y1:2768179.406
URGENT_ADDRESS2:台北市樂業街116號圍牆及118巷2~8號
X2:305689.109
Y2:2768179.406


In [ ]:
    
def download_data():
    conn = sqlite3.connect("./TPEroad.db")
    url = 'https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Bill_code TEXT,
                RCVdate TEXT,
                Start_date TEXT,
                End_date TEXT,
                Address1 TEXT,
                X1 TEXT,
                Y1 TEXT,
                Address2 TEXT,
                X2 TEXT,
                Y2 TEXT,
                UNIQUE(Address1,RCVdate)
            )
            ''')
            print("Table 'records' created or already exists.")
    except Exception as e:
        print(e)

    else:
        with conn:
            cursor = conn.cursor()
            for i in response_json['features']:
                for k in i['properties']:
                    Bill_code:str = i['properties']["BILL_CODE"]
                    RCVdate = i['properties']["URGENT_RCV_DATE"]
                    Start_date = i['properties']["URGENT_START_DATE"]
                    End_date = i['properties']["URGENT_END_DATE"]
                    Address1 = i['properties']["URGENT_ADDRESS1"]
                    X1 = i['properties']["X1"]
                    Y1 = i['properties']["Y1"]
                    Address2 = i['properties']["URGENT_ADDRESS2"]
                    X2 = i['properties']["X2"]
                    Y2 = i['properties']["Y2"]
                    
                    cursor = conn.cursor()
                    sql = '''INSERT OR IGNORE INTO records(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2)
                                values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                            '''
                    cursor.execute(sql,(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2))

現在有每一個x,y轉換成的地址 再把地址存成我用的資料表 再從資料表裡面提取我需要的區的位置 就可以做combobox的下拉式選單

In [ ]:
import functoins
import sqlite3
x = 305689.109
y = 2768207.525
address = functoins.latlonturn('reverse',x,y)


執行反轉操作
TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518
單次轉換結果['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號']
總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號']]


In [ ]:
import functoins
import sqlite3

conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()

    sql = '''
        SELECT Address1,x1,y1 FROM records
    '''

    cursor.execute(sql)
    # 把地址跟x1y1座標存進一個字典裡面
    address = [ ]
    for i in cursor.fetchall():
        di = {}
        di["地址"] = i[0]
        di["x1"] = i[1]
        di["y1"] = i[2]
        address.append(di)
    cursor.close() #把cursor關掉 清出記憶體
    print(address)

[{'地址': '台北市樂業街108巷1至11號', 'x1': '305689.109', 'y1': '2768179.406'}, {'地址': '士林區大東路１７６號', 'x1': '302989.777', 'y1': '2776097.833'}, {'地址': '信義區松壽路12號', 'x1': '307061.758', 'y1': '2769860.372'}, {'地址': '萬華區康定路２８８號', 'x1': '300605.315', 'y1': '2769933.376'}, {'地址': '信義區松智路１７號  松壽路12號', 'x1': '307058.453', 'y1': '2769555.577'}, {'地址': '萬華區西寧南路１２５號', 'x1': '301095.806', 'y1': '2770779.383'}, {'地址': '大安區仁愛路四段１１８號', 'x1': '305525.047', 'y1': '2770056.092'}, {'地址': '信義區忠孝東路五段６８之１號', 'x1': '307141.67101572', 'y1': '2770343.16104706'}, {'地址': '士林區永平街62號至78號', 'x1': '300949.784', 'y1': '2775880.777'}, {'地址': '信義區松壽路１３號', 'x1': '307284.192', 'y1': '2769877.164'}, {'地址': '信義區松智路１７號   松壽路12號', 'x1': '307262.652', 'y1': '2769660.839'}, {'地址': '信義區永吉路３２６巷４號至５０號', 'x1': '308066.079', 'y1': '2770843.205'}, {'地址': '中山區南京西路１５號', 'x1': '302616.632', 'y1': '2771686.983'}, {'地址': '南港區成福路２０８號', 'x1': '309455.969', 'y1': '2770315.641'}, {'地址': '萬華區桂林路1號(桂林路26號對面)', 'x1': '300988.69', 'y1': '2770062.864'}, {'地

接下來要提取每個字典裏面的x1y1然後丟到我的latlonturn裡面轉成地址(['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號'])  
再for i in list if '區' in i sql='insert ? into records' execute(sql,(i))
